In [7]:
import pickle, os
import xgboost
final_model = pickle.load(open(os.path.join('..', "models", "xgb_final.sav"), 'rb'))

In [8]:
from azureml.core import Workspace
ws = Workspace.from_config(path = os.path.join('..', 'config', 'config.json'))
print(ws)

Workspace.create(name='MSA-Phase2-Model', subscription_id='98860fb8-bddb-408a-8946-5bf9c599f8f7', resource_group='MSA-Phase-2-Model-Deployment')


In [10]:
from azureml.core.model import Model

# model = Model.register(ws, model_name = "lit_xgboost", model_path='xgb_final.sav')
model = Model(ws, id='lit_xgboost:1')

In [9]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

 
env = Environment(name = "xgboost")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("xgboost")
conda_dep.add_conda_package("nltk")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("pandas")
env.python.conda_dependencies = conda_dep

dummy_inference_config = InferenceConfig(
    environment = env,
    source_directory = "../source_dir",
    entry_script = "entry.py",
)

In [11]:
from azureml.core.webservice import AciWebservice
# NOTE: You may need more CPU or memeory depending on what kinds of model you end up using
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(
    ws,
    "xgboost",
    [model],
    dummy_inference_config,
    aci_config,
    overwrite = True,
)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-20 15:33:55+12:00 Creating Container Registry if not exists.
2021-09-20 15:33:55+12:00 Registering the environment.
2021-09-20 15:33:57+12:00 Generating deployment configuration.
2021-09-20 15:33:57+12:00 Use the existing image.
2021-09-20 15:33:58+12:00 Submitting deployment to compute.
2021-09-20 15:34:06+12:00 Checking the status of deployment xgboost..
2021-09-20 15:36:38+12:00 Checking the status of inference endpoint xgboost.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
import requests
import json
import pandas as pd
uri = service.scoring_uri
requests.get(uri)
headers = {"Content-Type": "application/json"}
data = pd.DataFrame.from_dict({"excerpt": ['hi', " It's not easy to be honest all the time.", "I was born", "I hurt someone", 'Emily, a lonely 32 year old woman,from the small town of Bethlehem, Pennsylvania, who grew up with her parents']})

data = data.to_json()

response = requests.post(uri, data = data, headers = headers)
print(response.json())

[-1.062340497970581, -1.1100631952285767, -1.062340497970581, -1.1092218160629272, -1.3607600927352905]
